# Extract – Transform – Load

## 1. Setting Up Spark Context

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

## 2. Download and Data from Kaggle

You need an API Token for Kaggle to work. Create one in your Kaggle account settings page and upload `kaggle.json`.

In [3]:
import os
import getpass

def get_or_set_environment_variable(variable):
    try:
        var = os.environ[variable]
    except KeyError:
        var = getpass.getpass('Please enter value for {:}: '.format(variable))
    
    os.environ[variable] = var
    return var

ibm_api_key_id = get_or_set_environment_variable('IBM_API_KEY_ID')
ibm_cloud_store_bucket = get_or_set_environment_variable('IBM_OBJECT_STORE_BUCKET')

Please enter value for IBM_API_KEY_ID: ········
Please enter value for IBM_OBJECT_STORE_BUCKET: ········


In [4]:
import json
import os

import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=ibm_api_key_id,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client.get_object(Bucket=ibm_cloud_store_bucket,
                         Key='kaggle.json')['Body']
# add missing __iter__ method

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

creds = json.load(body)
os.environ['KAGGLE_USERNAME'] = creds['username']
os.environ['KAGGLE_KEY'] = creds['key']

In [5]:
!kaggle competitions download -c nlp-getting-started -p "./work"

nlp-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
import glob

work_path = os.path.join(os.path.curdir, 'work')

zip_path = glob.glob(os.path.join(work_path, '*.zip'))[0]
os.path.getsize(zip_path)

607343

In [7]:
import zipfile

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(work_path)

In [8]:
glob.glob(os.path.join(work_path, '*.csv'))

['./work/train.csv', './work/sample_submission.csv', './work/test.csv']

## 3. Loading the files as `DataFrames` and splitting of the `'target'` column

In [9]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([StructField('id', IntegerType(), nullable=False),
                     StructField('keyword', StringType(), nullable=True),
                     StructField('location', StringType(), nullable=True),
                     StructField('text', StringType(), nullable=False)]
                   )
schema

StructType(List(StructField(id,IntegerType,false),StructField(keyword,StringType,true),StructField(location,StringType,true),StructField(text,StringType,false)))

In [10]:
df_test = spark.read.csv(os.path.join(work_path, 'test.csv'),
                         header=True,
                         encoding='UTF-8',
                         enforceSchema=True,
                         multiLine=True,
                         schema=schema,
                         mode='FAILFAST')
df_test.printSchema()
df_test.limit(10).toPandas()

root
 |-- id: integer (nullable = true)
 |-- keyword: string (nullable = true)
 |-- location: string (nullable = true)
 |-- text: string (nullable = true)



,id,keyword,location,text
0,0,None,None,Just happened a terrible car crash
1,2,None,None,"Heard about #earthquake is different cities, s..."
2,3,None,None,"there is a forest fire at spot pond, geese are..."
3,9,None,None,Apocalypse lighting. #Spokane #wildfires
4,11,None,None,Typhoon Soudelor kills 28 in China and Taiwan
5,12,None,None,We're shaking...It's an earthquake
6,21,None,None,They'd probably still show more life than Arse...
7,22,None,None,Hey! How are you?
8,27,None,None,What a nice hat?
9,29,None,None,Fuck off!


In [11]:
schema.add(StructField('target', IntegerType(), nullable=False))
schema

StructType(List(StructField(id,IntegerType,false),StructField(keyword,StringType,true),StructField(location,StringType,true),StructField(text,StringType,false),StructField(target,IntegerType,false)))

In [12]:
df_train = spark.read.csv(os.path.join(work_path, 'train.csv'),
                          header=True,
                          encoding='UTF-8',
                          multiLine=True,
                          enforceSchema=True,
                          schema=schema,
                          mode='FAILFAST')
df_train.printSchema()
df_train.limit(10).toPandas()

root
 |-- id: integer (nullable = true)
 |-- keyword: string (nullable = true)
 |-- location: string (nullable = true)
 |-- text: string (nullable = true)
 |-- target: integer (nullable = true)



,id,keyword,location,text,target
0,1,None,None,Our Deeds are the Reason of this #earthquake M...,1
1,4,None,None,Forest fire near La Ronge Sask. Canada,1
2,5,None,None,All residents asked to 'shelter in place' are ...,1
3,6,None,None,"13,000 people receive #wildfires evacuation or...",1
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...,1
5,8,None,None,#RockyFire Update => California Hwy. 20 closed...,1
6,10,None,None,#flood #disaster Heavy rain causes flash flood...,1
7,13,None,None,I'm on top of the hill and I can see a fire in...,1
8,14,None,None,There's an emergency evacuation happening now ...,1
9,15,None,None,I'm afraid that the tornado is coming to our a...,1


In [13]:
df_label = df_train.select('id', 'target')
df_label.printSchema()
df_label.limit(10).toPandas()

root
 |-- id: integer (nullable = true)
 |-- target: integer (nullable = true)



,id,target
0,1,1
1,4,1
2,5,1
3,6,1
4,7,1
5,8,1
6,10,1
7,13,1
8,14,1
9,15,1


In [14]:
df_train = df_train.drop('target')
df_train.limit(10).toPandas()

,id,keyword,location,text
0,1,None,None,Our Deeds are the Reason of this #earthquake M...
1,4,None,None,Forest fire near La Ronge Sask. Canada
2,5,None,None,All residents asked to 'shelter in place' are ...
3,6,None,None,"13,000 people receive #wildfires evacuation or..."
4,7,None,None,Just got sent this photo from Ruby #Alaska as ...
5,8,None,None,#RockyFire Update => California Hwy. 20 closed...
6,10,None,None,#flood #disaster Heavy rain causes flash flood...
7,13,None,None,I'm on top of the hill and I can see a fire in...
8,14,None,None,There's an emergency evacuation happening now ...
9,15,None,None,I'm afraid that the tornado is coming to our a...


## 4. Serializing the dataframes in *Parquet* format

In [15]:
!rm -r ./disaster_detection_*

rm: cannot remove './disaster_detection_*': No such file or directory


In [16]:
temp_parquet_file = os.path.join(os.path.curdir,
                                 'disaster_detection_{}')
df_train.write.parquet(temp_parquet_file.format('train'))
df_label.write.parquet(temp_parquet_file.format('label'))
df_test.write.parquet(temp_parquet_file.format('test'))

glob.glob(temp_parquet_file.format('*'))

['./disaster_detection_train',
 './disaster_detection_label',
 './disaster_detection_test']

## 5. Uploading the files to object cloud

In [17]:
def upload_parquet(client, path):
    parts = glob.glob(os.path.join(path, '*.parquet'))
    parquets = ['{:s}-{:04d}.parquet'.format(os.path.split(path)[-1], i)
                for i in range(len(parts))]
    for part, parquet in zip(parts, parquets):
        with open(part, 'rb') as parquetF:
            client.put_object(Bucket=ibm_cloud_store_bucket,
                          Body=parquetF,
                          Key=parquet
                         )
    return parquets

client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=ibm_api_key_id,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')


parquets = {}
for dataset in ('train', 'label', 'test'):
    parquets[dataset] = upload_parquet(client, temp_parquet_file.format(dataset))

print(parquets)

{'train': ['disaster_detection_train-0000.parquet'], 'label': ['disaster_detection_label-0000.parquet'], 'test': ['disaster_detection_test-0000.parquet']}


In [18]:
import json

client.put_object(Bucket=ibm_cloud_store_bucket,
                  Body=json.dumps(parquets),
                  Key='etl_parquet_files.json')

{'ResponseMetadata': {'RequestId': 'f9233808-ec22-46b4-ace7-75f069a9f528',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 06 Feb 2021 21:42:04 GMT',
   'x-clv-request-id': 'f9233808-ec22-46b4-ace7-75f069a9f528',
   'server': 'Cleversafe',
   'x-clv-s3-version': '2.5',
   'x-amz-request-id': 'f9233808-ec22-46b4-ace7-75f069a9f528',
   'etag': '"71f41b034280b8afc049f8a95a788d25"',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"71f41b034280b8afc049f8a95a788d25"'}